In [7]:
import pandas as pd
import numpy as np

In [8]:
import pickle

In [9]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [10]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi")

<Experiment: artifact_location='/home/estevam/dataside/mlops-zoomcamp/02 - Experiment Tracking/mlruns/1', creation_time=1723481053030, experiment_id='1', last_update_time=1723481053030, lifecycle_stage='active', name='nyc-taxi', tags={}>

In [11]:
def pre_processing(path_df: str) -> pd.DataFrame:
    # Load DataFrame
    df = pd.read_parquet(path=path_df)
    
    if 'lpep_pickup_datetime' in df.columns and 'lpep_dropoff_datetime' in df.columns:
        pickup_col, dropoff_col = 'lpep_pickup_datetime', 'lpep_dropoff_datetime'
    elif 'tpep_pickup_datetime' in df.columns and 'tpep_dropoff_datetime' in df.columns:
        pickup_col, dropoff_col = 'tpep_pickup_datetime', 'tpep_dropoff_datetime'
    
    # Transformando valores em datetime e criando a coluna duração
    df[pickup_col] = pd.to_datetime(df[pickup_col])
    df[dropoff_col] = pd.to_datetime(df[dropoff_col])
    df['duration'] = df[dropoff_col] - df[pickup_col]
    df['duration'] = df['duration'].apply(lambda time: time.total_seconds() / 60)
    
    # Filtrando Duração da Viagem
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
    
    return df

In [12]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

def transform_variables(
    df: pd.DataFrame,
    dict_vectorizer: DictVectorizer,
):
    # Transformar variáveis categóricas
    df[categorical] = df[categorical].astype(str)

    data_dicts = df[categorical + numerical].to_dict(orient="records")
    X = dict_vectorizer.transform(data_dicts)
    y = df["duration"].values

    return X, y

In [13]:
df_train = pre_processing(
    path_df="data/green_tripdata_2021-01.parquet"
)  # janeiro
df_test = pre_processing(
    path_df="data/green_tripdata_2021-02.parquet"
)  # fevereiro

In [14]:
dict_vectorizer = DictVectorizer()

# Transformação do conjunto de treinamento
df_train[categorical] = df_train[categorical].astype(str)
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dict_vectorizer.fit_transform(train_dicts)
y_train = df_train['duration'].values

# Transformação do conjunto de teste usando o mesmo DictVectorizer
X_test, y_test = transform_variables(df_test, dict_vectorizer)

In [15]:
with mlflow.start_run():
    mlflow.set_tag("developer", "estevam")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
    mlflow.log_param("model", "linear-regression")
    
    
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_test)
    rmse = mean_squared_error(y_test, lr.predict(X_test), squared=False)
    mlflow.log_metric("train-mse", rmse)
    
    mlflow.log_artifact(local_path="./models/lin_reg.bin", artifact_path="models_pickle")

/home/estevam/miniconda3/envs/zoomcamp/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [16]:
with mlflow.start_run():
    mlflow.set_tag("developer", "estevam")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
    mlflow.log_param("model", "lasso")
    
    lr = Lasso()
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_test)
    rmse = mean_squared_error(y_test, lr.predict(X_test), squared=False)
    mlflow.log_metric("train-mse", rmse)
    
    mlflow.log_artifact(local_path="./models/lin_reg.bin", artifact_path="models_pickle")

/home/estevam/miniconda3/envs/zoomcamp/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [17]:
import xgboost as xgb

In [18]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [20]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_test, label=y_test)

In [21]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [22]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/estevam/miniconda3/envs/zoomcamp/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [16:45:16] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.22499                          
[1]	validation-rmse:8.91365                           
[2]	validation-rmse:8.07795                           
[3]	validation-rmse:7.54392                           
[4]	validation-rmse:7.21195                           
[5]	validation-rmse:7.00266                           
[6]	validation-rmse:6.86921                           
[7]	validation-rmse:6.77625                           
[8]	validation-rmse:6.71959                           
[9]	validation-rmse:6.66575                           
[10]	validation-rmse:6.63092                          
[11]	validation-rmse:6.60765                          
[12]	validation-rmse:6.58497                          
[13]	validation-rmse:6.57021                          
[14]	validation-rmse:6.55508                          
[15]	validation-rmse:6.54502                          
[16]	validation-rmse:6.53315                          
[17]	validation-rmse:6.52440                          
[18]	valid

/home/estevam/miniconda3/envs/zoomcamp/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/estevam/miniconda3/envs/zoomcamp/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [16:45:27] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[2]	validation-rmse:10.37135                                                   
[3]	validation-rmse:9.88769                                                    
[4]	validation-rmse:9.45851                                                    
[5]	validation-rmse:9.07883                                                    
[6]	validation-rmse:8.74536                                                    
[7]	validation-rmse:8.45156                                                    
[8]	validation-rmse:8.19454                                                    
[9]	validation-rmse:7.96753                                                    
[10]	validation-rmse:7.76993                                                   
[11]	validation-rmse:7.59734                                                   
[12]	validation-rmse:7.44558                                                   
[13]	validation-rmse:7.31235                                                   
[14]	validation-rmse:7.19527            

...

/home/estevam/miniconda3/envs/zoomcamp/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/estevam/miniconda3/envs/zoomcamp/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [16:51:17] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[3]	validation-rmse:6.49357                                                     
[4]	validation-rmse:6.46807                                                     
[5]	validation-rmse:6.44689                                                     
[6]	validation-rmse:6.42938                                                     
[7]	validation-rmse:6.40151                                                     
[8]	validation-rmse:6.38678                                                     
[9]	validation-rmse:6.38025                                                     
[10]	validation-rmse:6.37501                                                    
[11]	validation-rmse:6.36097                                                    
[12]	validation-rmse:6.35662                                                    
[13]	validation-rmse:6.35152                                                    
[14]	validation-rmse:6.34573                                                    
[15]	validation-rmse:6.34171

/home/estevam/miniconda3/envs/zoomcamp/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/estevam/miniconda3/envs/zoomcamp/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [16:51:21] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.76043                                                     
[1]	validation-rmse:6.73173                                                     
[2]	validation-rmse:6.48984                                                     
[3]	validation-rmse:6.41642                                                     
[4]	validation-rmse:6.38520                                                     
[5]	validation-rmse:6.36502                                                     
[6]	validation-rmse:6.35804                                                     
[7]	validation-rmse:6.35686                                                     
[8]	validation-rmse:6.35906                                                     
[9]	validation-rmse:6.36104                                                     
[10]	validation-rmse:6.36145                                                    
[11]	validation-rmse:6.35731                                                    
[12]	validation-rmse:6.36146

/home/estevam/miniconda3/envs/zoomcamp/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

